<a href="https://colab.research.google.com/github/Priyanka-Police-Reddy-Gari/EncoderModelsComparition/blob/main/Priyanka_PoliceReddyGari_HW_6_Part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  from google.colab import drive
  # Mount the Google Drive to access files stored there
  drive.mount('/content/drive')

  # Install the latest version of torchtext library quietly without showing output
  !pip install torchtext -qq
  !pip install transformers evaluate wandb datasets accelerate -U -qq ## NEW LINES ##
  basepath = '/content/drive/MyDrive/'

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path


# New libraries introduced in this notebook
import evaluate
from datasets import load_dataset, DatasetDict, Dataset, Value, ClassLabel, Features
from transformers import TrainingArguments, Trainer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from transformers import AutoTokenizer, DistilBertTokenizer
from transformers import AutoModelForSequenceClassification,  DistilBertModel, AutoModelForSeq2SeqLM
from transformers import AutoConfig, DistilBertConfig
from transformers import pipeline
from transformers import EvalPrediction
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import wandb
import torch

In [ ]:
base_folder = Path(basepath)
data_folder = base_folder/'NLP/Assignment_5'
model_folder = base_folder/'NLP/Assignment_5/models'
custom_functions = base_folder/'data/custom-functions'

In [ ]:
base_folder

PosixPath('/content/drive/MyDrive')

In [ ]:
data_folder

PosixPath('/content/drive/MyDrive/NLP/Assignment_5')

# Load Data

In [ ]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/NLP/Assignment_5/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/NLP/Assignment_5/test.csv')

In [ ]:
train_df.head()

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2017-21441,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,2017-31535,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,2017-21068,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,2017-31436,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,2017-22195,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0


In [ ]:
test_df.head()

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,@Adnan__786__ @AsYouNotWish Dont worry Indian ...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
1,2018-03739,"Academy of Sciences, eschews the normally sobe...",NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
2,2018-00385,I blew that opportunity -__- #mad,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
3,2018-03001,This time in 2 weeks I will be 30... 😥,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
4,2018-01988,#Deppression is real. Partners w/ #depressed p...,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE


In [ ]:
labels = [label for label in train_df.columns if label not in ["Tweet","ID"]]
id2labels = {idx:label for idx, label in enumerate(labels)}
labels2id = {v:k for k,v in id2labels.items()}
labels2id

{'anger': 0,
 'anticipation': 1,
 'disgust': 2,
 'fear': 3,
 'joy': 4,
 'love': 5,
 'optimism': 6,
 'pessimism': 7,
 'sadness': 8,
 'surprise': 9,
 'trust': 10}

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_set, valid_set = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:
train_text = train_set['Tweet'].tolist()
train_label = train_set.drop(columns=['ID','Tweet']).to_numpy().astype(float)
valid_text = valid_set['Tweet'].tolist()
valid_label = valid_set.drop(columns=['ID','Tweet']).to_numpy().astype(float)

In [ ]:
train_text

["Going to get myself a copy of @StephenKing's CUJO for an upcoming project that I can't talk about just yet.  #amwriting",
 "@carysmithwriter @Maria_Savva @RealRockAndRoll We're the least known band in the World, but so glad you asked #muchlove ",
 'Unruly kids at 8am in the morning #nothanks ripping the flower beds up by the roots while their parents watch #shocking',
 "Ok but I just got called a 'White Devil' on the train and I didnt know whether to laugh or be offended",
 '@SXMUrbanView @karenhunter @CousinSyl you are so wrong for this!needed levity after that recording',
 "Even a pencil✏ never #stayed  with me until it's #end ⚫ 😞",
 "@pottermore : I can't find my patronus, the website doesn't work, I can't even see the questions.... #sadness...",
 "Pride and Prejudice is a modern day Keeping up with the Kardashians' - @howsyourdaybin .............I've never been so offended in my life",
 'Wish I could convince the rest of my children to go to their fucking preschool and elementary

In [ ]:
train_label

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 1., 0.],
       ...,
       [1., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
test_text = test_df['Tweet'].tolist()
test_label = test_df.drop(columns=['ID','Tweet']).to_numpy()
test_label = np.where(test_label == "NONE",0,test_label).astype(float)

In [ ]:
test_text

['@Adnan__786__ @AsYouNotWish Dont worry Indian army is on its ways to dispatch all Terrorists to Hell',
 'Academy of Sciences, eschews the normally sober tone of scientific papers and calls the massive loss of wildlife a “biological annihilation',
 'I blew that opportunity -__- #mad',
 'This time in 2 weeks I will be 30... 😥',
 '#Deppression is real. Partners w/ #depressed people truly dont understand the depth in which they affect us. Add in #anxiety &amp;makes it worse',
 '@POLITICOEurope Interesting choice of words... Are you confirming that governments fund #terrorism? Bit of an open door, but still...',
 '@ananavarro CNN should, for sure 😀',
 'Distance yourself once stretched by your friends impose! #serious #loveyou #notseriously',
 'Be happy. Be confident. Be kind.\\n\\n #KissablesLoveSMShopmag\\nAllOutDenimFor KISSMARC',
 'My visit to hospital for care triggered #trauma from accident 20+yrs ago and image of my dead brother in it. Feeling symptoms of #depression',
 "Thanks to D

In [ ]:
test_label

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
(len(train_text), train_label.shape, len(valid_text), valid_label.shape, len(test_text), test_label.shape)

(6179, (6179, 11), 1545, (1545, 11), 3259, (3259, 11))

In [ ]:
type(train_text)

list

# Create Huggingface DataSet

In [ ]:
trainset = Dataset.from_dict({
    'texts': train_text,
    'labels': train_label
})

validset = Dataset.from_dict({
    'texts': valid_text,
    'labels': valid_label
})

testset = Dataset.from_dict({
    'texts': test_text,
    'labels': test_label
})

In [ ]:
trainset

Dataset({
    features: ['texts', 'labels'],
    num_rows: 6179
})

In [ ]:
trainset.features

{'texts': Value(dtype='string', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)}

In [ ]:
# trainset.features['labels'] = ClassLabel(num_classes=11, names=['anger','anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust'])
# trainset = trainset.cast(trainset.features)

In [ ]:
trainset.features['labels']

Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)

In [ ]:
trainset

Dataset({
    features: ['texts', 'labels'],
    num_rows: 6179
})

In [ ]:
trainset.features

{'texts': Value(dtype='string', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None)}

In [ ]:
trainset[1]

{'texts': "@carysmithwriter @Maria_Savva @RealRockAndRoll We're the least known band in the World, but so glad you asked #muchlove ",
 'labels': [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]}

In [ ]:
trainset[0]['texts']

"Going to get myself a copy of @StephenKing's CUJO for an upcoming project that I can't talk about just yet.  #amwriting"

In [ ]:
trainset[1]['labels']

[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]

# Load pre-trained Tokenizer

In [ ]:
checkpoint = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

# Create function for Tokenizer

In [ ]:
max_length = 128

In [ ]:
def tokenize_fn(batch):
    return tokenizer(text = batch["texts"], truncation=True, padding="max_length", return_tensors="pt",max_length = 128)

In [ ]:
tokenized_traindataset= trainset.map(tokenize_fn, batched=True)
tokenized_validdataset= validset.map(tokenize_fn, batched=True)
tokenized_testdataset= testset.map(tokenize_fn, batched=True)

Map:   0%|          | 0/6179 [00:00<?, ? examples/s]

Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [ ]:
tokenized_traindataset

Dataset({
    features: ['texts', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 6179
})

In [ ]:
tokenized_traindataset = tokenized_traindataset.remove_columns(['texts'])
tokenized_validdataset = tokenized_validdataset.remove_columns(['texts'])
tokenized_testdataset = tokenized_testdataset.remove_columns(['texts'])

In [ ]:
tokenized_traindataset.set_format(type='torch')
tokenized_validdataset.set_format(type='torch')
tokenized_testdataset.set_format(type='torch')

In [ ]:
tokenized_traindataset[1]

{'labels': tensor([0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.]),
 'input_ids': tensor([ 3320,    75,  1208, 16331, 12756,  3320,   329,  6286,   834,   134,
             9,   208,   900,  3320,  1649,   138, 23349,  7175, 29807,   101,
            31,    60,     8,   709,   801,  1928,    16,     8,  1150,     6,
            68,    78,  3755,    25,  1380,  1713,    51,  2295,  5850,    15,
             3,     1,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,   

In [ ]:
tokenized_traindataset.features

{'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

# Model Training

## Download pre-trained model

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = 11,
                                              problem_type = "multi_label_classification", id2label = id2labels , label2id = labels2id
                                              )

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Compute Metrics

In [ ]:
# def combined_metrics(predictions, labels, threshold=0.5):
#     probs = torch.sigmoid(torch.Tensor(predictions))
#     y_pred = (probs >= threshold).int()
#     metrics = {'f1': f1_score(y_true=labels, y_pred=y_pred, average='macro'),
#             'Accuracy': accuracy_score(labels, y_pred)}
#     # combined_metrics = evaluate.combine([evaluate.load("accuracy", ),
#     #                                     evaluate.load("f1", average="micro")])
#     return metrics

# def compute_metrics(p: EvalPrediction):
#     predictions, labels = (p.predictions, p.label_ids) if isinstance(p.predictions, tuple) else (p.predictions, p.label_ids)
#     return combined_metrics(predictions=predictions[0], labels=labels)

In [ ]:
#sklearn
def metrics(predictions, labels, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    y_true = labels
    f1_micro = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    f1_macro = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    accuracy = accuracy_score(y_true, y_pred)
    metrics = {'f1_micro': f1_micro,'f1_macro': f1_macro,'Accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,tuple) else p.predictions
    result = metrics(predictions=preds,labels=p.label_ids)
    return result

In [ ]:
# #evaluate
# accuracy_metric = evaluate.load('accuracy', 'multilabel')
# f1 = evaluate.load('f1','multilabel')


# def compute_metrics(eval_pred):
#     # accuracy_metric = evaluate.load('accuracy', 'multilabel')

#     logits, labels = eval_pred
#     # print(logits.shape)
#     preds = (logits > 0).astype(tuple)
#     accuracy = accuracy_metric.compute(predictions=preds, references=labels)
#     f1_micro = f1.compute(predictions=preds, references=labels, average='micro')
#     f1_macro = f1.compute(predictions=preds, references=labels, average='macro')
#     return {'f1_micro':f1_micro['f1'],
#             'f1_macro':f1_macro['f1'],
#             'accuracy':accuracy['accuracy'],
#             }

In [ ]:
# def compute_metrics(eval_pred):
#     combined_metrics = evaluate.combine([evaluate.load("accuracy",'multilabel' ),
#                                         evaluate.load("f1", average="micro",'multilabel')])

#     logits, labels = eval_pred
#     predictions = (logits > 0).astype(int)
#     # predictions = (logits > 0).astype(tuple).reshape(-1)
#     f1_micro = f1.compute(predictions=preds, references=labels.astype(int), average='micro')
#     f1_macro = f1.compute(predictions=preds, references=labels.astype(int), average='macro')
#     accuracy = accuracy.compute(predictions=predictions, references=labels.astype(int))
#     evaluations = combined_metrics.compute(
#         predictions=predictions, references=labels.astype(int))
#     return evaluations
#     #return { 'f1_micro': f1_micro['f1'], 'f1_macro': f1_macro['f1'], 'accuracy': accuracy['accuracy']}

## Train Args

In [ ]:
# Define the directory where model checkpoints will be saved
run_name = "HW6/flan-t5-base"
base_folder = Path(basepath)
model_folder = base_folder / "models"/run_name
# Create the directory if it doesn't exist
model_folder.mkdir(exist_ok=True, parents=True)

# Configure training parameters
training_args = TrainingArguments(

    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,  # Apply L2 regularization to prevent overfitting
    learning_rate=0.00001,
    lr_scheduler_type= 'linear',# Step size for the optimizer during training
    optim='adamw_torch',  # Optimizer,
    remove_unused_columns=False,  # Remove unused columns from the dataset
    output_dir=str(model_folder),# Directory to save model checkpoints
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=50,  # Perform evaluation every 10 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=50,  # Save a model checkpoint every 10 training steps
    load_best_model_at_end=True,  # Reload the best model at the end of training
    save_total_limit=2,  # Retain only the best and the most recent model checkpoints
    # Use 'accuracy' as the metric to determine the best model
    metric_for_best_model="eval_f1_macro",
    greater_is_better=True,  # A model is 'better' if its accuracy is higher


    # Experiment logging configurations (commented out in this example)
    logging_strategy='steps',
    logging_steps=20,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name=run_name,  # Experiment name for Weights & Biases

    fp16=False,
    bf16=False,
    tf32=False,

)


In [ ]:
# data_collator = DataCollatorForSeq2Seq(tokenizer,
#                                        model=model,
#                                        pad_to_multiple_of = 8,
#                                        label_pad_token_id = labels2id
#                                        )

## Initialize Trainer

In [ ]:
# initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_traindataset,
    eval_dataset=tokenized_validdataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Setup wand

In [ ]:
!wandb login e2d6f3f082a17b4b9828163a71499e33f6d9bfd4
%env WANDB_PROJECT = run_name

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
env: WANDB_PROJECT=run_name


## Training and Validation

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,F1 Micro,F1 Macro,Accuracy
50,0.521300,0.496377,0.023637,0.012010,0.022654
100,0.496600,0.477147,0.003281,0.001829,0.023301
150,0.485800,0.474465,0.008186,0.004477,0.023948
200,0.484200,0.472845,0.006011,0.003338,0.024595
250,0.465500,0.471286,0.016291,0.009192,0.024595
300,0.458000,0.469015,0.030620,0.016686,0.023948
350,0.459000,0.465541,0.058916,0.030716,0.023948
400,0.463600,0.460696,0.137457,0.058589,0.029773
450,0.441900,0.453867,0.146870,0.071803,0.033657
500,0.450400,0.447474,0.189952,0.092296,0.040129


There were missing keys in the checkpoint model loaded: ['transformer.encoder.embed_tokens.weight', 'transformer.decoder.embed_tokens.weight'].


TrainOutput(global_step=3865, training_loss=0.3835679935452855, metrics={'train_runtime': 2460.1787, 'train_samples_per_second': 12.558, 'train_steps_per_second': 1.571, 'total_flos': 4717652405948160.0, 'train_loss': 0.3835679935452855, 'epoch': 5.0})

Evaluate model on Validation Set

In [ ]:
eval_results = trainer.evaluate(tokenized_validdataset)

In [ ]:
eval_results

{'eval_loss': 0.3542534112930298,
 'eval_f1_micro': 0.6123899273907,
 'eval_f1_macro': 0.37532688017892807,
 'eval_Accuracy': 0.18705501618122977,
 'eval_runtime': 11.3905,
 'eval_samples_per_second': 135.639,
 'eval_steps_per_second': 17.032,
 'epoch': 5.0}

In [ ]:
wandb.log({"eval_accuracy": eval_results["eval_Accuracy"], "eval_loss": eval_results["eval_loss"], "eval_f1_micro": eval_results["eval_f1_micro"], "eval_f1_macro": eval_results["eval_f1_macro"]})

Check best points

In [ ]:
# After training, let us check the best checkpoint
# We need this for Inference
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 3350.


# Performance on Test set

In [ ]:
checkpoint = str(model_folder/f'checkpoint-{best_model_checkpoint_step}')
checkpoint

'/content/drive/MyDrive/models/HW6/flan-t5-base/checkpoint-3350'

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir=str(model_folder),
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to='wandb',
    run_name=run_name
)

Instantiate Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=tokenized_testdataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Evaluate using Trainer

In [ ]:
test_results= trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [ ]:
test_results

{'eval_loss': 0.40258005261421204,
 'eval_f1_micro': 0.0,
 'eval_f1_macro': 0.0,
 'eval_Accuracy': 0.08315434182264499,
 'eval_runtime': 26.5729,
 'eval_samples_per_second': 122.644,
 'eval_steps_per_second': 15.354}

In [ ]:
wandb.log({"test_accuracy": test_results["eval_Accuracy"], "test_loss": test_results["eval_loss"], "eval_f1_micro": test_results["eval_f1_micro"], "eval_f1_macro": test_results["eval_f1_macro"]})

In [ ]:
test_output = trainer.predict(tokenized_testdataset)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [ ]:
test_preds = test_output.predictions
test_preds = torch.tensor(test_preds[0])
test_labels = np.array(test_output.label_ids)


In [ ]:
wandb.finish()

eval/Accuracy,▁▁▁▁▁▁▂▅▆▇▇▇▇▆▇▇▇▇▇▇█▇▇▇█▇▇████████████▄
eval/f1_macro,▂▁▁▁▂▂▃▅▅▆▆▆▇▇▇▇▇▇▇▇█▇█▇███████████████▁
eval/f1_micro,▂▁▁▁▂▃▄▆▆▇▇▇▇██████████████████████████▁
eval/loss,█▅▄▄▄▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
eval/runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
eval/samples_per_second,▄▅████▇▇▇▆▇▆▄▆▆▆▅▇▅▅▅▆▇▄▇▇▇▇▆▇▇▅▇▆▇▇▇▇▇▁
eval/steps_per_second,▄▅████▇▇▇▆▇▆▄▆▆▆▅▇▅▅▅▆▇▄▇▇▇▇▆▇▇▅▇▆▇▇▇▇▇▁
eval_accuracy,▁
eval_f1_macro,█▁
eval_f1_micro,█▁
eval_loss,▁


In [ ]:
test_output

PredictionOutput(predictions=(array([[ 1.0874041 , -2.849627  ,  0.95112026, ..., -0.17063619,
        -3.693845  , -4.577097  ],
       [ 0.42936713, -2.4209409 ,  0.5672507 , ..., -0.7180377 ,
        -3.1438167 , -3.9028275 ],
       [ 0.3281301 , -3.1422787 ,  0.85271835, ...,  0.81787366,
        -3.6392534 , -4.7010236 ],
       ...,
       [ 2.2548149 , -3.077288  ,  2.4592662 , ..., -0.8311406 ,
        -3.5277822 , -4.591065  ],
       [-3.7824526 , -1.7317543 , -3.6305513 , ..., -3.3105025 ,
        -2.5489209 , -2.0709019 ],
       [-3.3810177 , -1.8511772 , -3.3858426 , ..., -2.8381717 ,
        -2.7383258 , -2.8072054 ]], dtype=float32), array([[[-0.0591545 ,  0.1466862 , -0.10400037, ...,  0.09077623,
          0.11380879, -0.08462268],
        [-0.03368152,  0.14613841, -0.24829705, ...,  0.05499642,
          0.11016597, -0.19233243],
        [-0.07442155,  0.08308681, -0.14634143, ..., -0.01498263,
          0.01547946, -0.04738463],
        ...,
        [-0.10954517, 

In [ ]:
test_output.predictions

(array([[ 1.0874041 , -2.849627  ,  0.95112026, ..., -0.17063619,
         -3.693845  , -4.577097  ],
        [ 0.42936713, -2.4209409 ,  0.5672507 , ..., -0.7180377 ,
         -3.1438167 , -3.9028275 ],
        [ 0.3281301 , -3.1422787 ,  0.85271835, ...,  0.81787366,
         -3.6392534 , -4.7010236 ],
        ...,
        [ 2.2548149 , -3.077288  ,  2.4592662 , ..., -0.8311406 ,
         -3.5277822 , -4.591065  ],
        [-3.7824526 , -1.7317543 , -3.6305513 , ..., -3.3105025 ,
         -2.5489209 , -2.0709019 ],
        [-3.3810177 , -1.8511772 , -3.3858426 , ..., -2.8381717 ,
         -2.7383258 , -2.8072054 ]], dtype=float32),
 array([[[-0.0591545 ,  0.1466862 , -0.10400037, ...,  0.09077623,
           0.11380879, -0.08462268],
         [-0.03368152,  0.14613841, -0.24829705, ...,  0.05499642,
           0.11016597, -0.19233243],
         [-0.07442155,  0.08308681, -0.14634143, ..., -0.01498263,
           0.01547946, -0.04738463],
         ...,
         [-0.10954517,  0.392622

In [ ]:
preds = test_output.predictions

In [ ]:
import torch

In [ ]:
preds = torch.tensor(preds[0])

predictions to probabilty

In [ ]:
def conversion(predictions, threshold=0.25):
    probs = torch.sigmoid(torch.Tensor(predictions))
    return np.where(probs >= threshold, 1, 0)

In [ ]:
test_pred_output = conversion(preds)

# Model Inference

In [ ]:
checkpoint = str(model_folder/f'checkpoint-{best_model_checkpoint_step}')
custom_pipeline = pipeline(
    task="text-classification",
    model=checkpoint,
    tokenizer=checkpoint,
    device=0)

In [ ]:
map_labels = []
for predictions in test_pred_output:
    map_row = [label for label, pred in zip(labels, predictions) if pred == 1]
    map_labels.append(map_row)

In [ ]:
print(map_labels[0:5])

[['anger', 'disgust', 'fear', 'sadness'], ['anger', 'disgust', 'sadness'], ['anger', 'disgust', 'fear', 'sadness'], ['joy', 'sadness'], ['anger', 'disgust', 'fear', 'pessimism', 'sadness']]


In [ ]:
test_df[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
       'optimism', 'pessimism', 'sadness', 'surprise', 'trust']] = test_pred_output

In [ ]:
test_df.head()

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,@Adnan__786__ @AsYouNotWish Dont worry Indian ...,1,0,1,1,0,0,0,0,1,0,0
1,2018-03739,"Academy of Sciences, eschews the normally sobe...",1,0,1,0,0,0,0,0,1,0,0
2,2018-00385,I blew that opportunity -__- #mad,1,0,1,1,0,0,0,0,1,0,0
3,2018-03001,This time in 2 weeks I will be 30... 😥,0,0,0,0,1,0,0,0,1,0,0
4,2018-01988,#Deppression is real. Partners w/ #depressed p...,1,0,1,1,0,0,0,1,1,0,0


In [ ]:
test_df.drop(["Tweet"],axis =1, inplace = True)

In [ ]:
test_df.to_csv(data_folder/"flan-test.csv")